In [1]:
import os

In [2]:
%pwd

"c:\\Arjun's Work\\Chest-Cancer-Detection\\research"

In [3]:
os.chdir('../')

In [4]:
%pwd

"c:\\Arjun's Work\\Chest-Cancer-Detection"

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from chest_cancer_detection.constants import *
from chest_cancer_detection.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-scan-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [ ]:
from pathlib import Path
import tensorflow as tf
from sklearn.metrics import classification_report
import numpy as np

class Training:
    def __init__(self, config):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
            class_mode="categorical"  # Critical for softmax output
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

        # 🧪 Classification report
        print("\n📊 Generating classification report...")
        steps = self.valid_generator.samples // self.valid_generator.batch_size + 1
        y_pred_probs = self.model.predict(self.valid_generator, steps=steps)
        y_pred = np.argmax(y_pred_probs, axis=1)
        y_true = self.valid_generator.classes[:len(y_pred)]

        report = classification_report(
            y_true,
            y_pred,
            target_names=list(self.valid_generator.class_indices.keys())
        )
        print(report)

        # Optionally save the report to a file
        with open("classification_report.txt", "w") as f:
            f.write(report)


In [13]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-06-19 16:48:50,122: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-19 16:48:50,126: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-19 16:48:50,128: INFO: common: created directory at: artifacts]
[2025-06-19 16:48:50,129: INFO: common: created directory at: artifacts\training]
Found 96 images belonging to 2 classes.
Found 389 images belonging to 2 classes.
Epoch 1/3
24/24 [==============================] - 34s 1s/step - loss: 14.2756 - accuracy: 0.5416 - val_loss: 15.2213 - val_accuracy: 0.6771
Epoch 2/3
24/24 [==============================] - 33s 1s/step - loss: 9.5218 - accuracy: 0.6139 - val_loss: 1.3331 - val_accuracy: 0.8854
Epoch 3/3
24/24 [==============================] - 38s 2s/step - loss: 5.9802 - accuracy: 0.7161 - val_loss: 3.1543 - val_accuracy: 0.8021

📊 Generating classification report...
7/7 [==============================] - 7s 953ms/step
                precision    recall  f1-score   support

Adenocarcinoma       